<a href="https://colab.research.google.com/github/msuraj248/Keras_project/blob/master/CatsVSDogsUsingKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras.layers import Dense, Activation, Flatten
from keras.layers.convolutional import *
from keras.layers.pooling import *
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
tf.compat.v1.disable_eager_execution()

In [ ]:
os.chdir('D:/documents/CNN/train')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
    for c in random.sample(glob.glob('cat*'),500):
        shutil.move(c, 'train/cat')
    for c in random.sample(glob.glob('dog*'),500):
        shutil.move(c, 'train/dog')
    for c in random.sample(glob.glob('cat*'),100):
        shutil.move(c, 'valid/cat')
    for c in random.sample(glob.glob('dog*'),100):
        shutil.move(c, 'valid/dog')
    for c in random.sample(glob.glob('cat*'),50):
        shutil.move(c, 'test/cat')
    for c in random.sample(glob.glob('dog*'),50):
        shutil.move(c, 'test/dog')

In [2]:
train_path = 'drive/My Drive/CNN/train'
valid_path = 'drive/My Drive/CNN/valid'
test_path = 'drive/My Drive/CNN/test'

In [20]:
train_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input)\
                  .flow_from_directory(directory= train_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input)\
                  .flow_from_directory(directory= valid_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input)\
                  .flow_from_directory(directory= test_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size=10, shuffle = False)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [21]:
imgs, lables = next(train_batches)

In [22]:
def plot_image(images):
  fig, axes = plt.subplots(1,10, figsize= (20,20))
  axes = axes.flatten()
  for img, ax in zip(images, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plot_image(imgs)
print(lables)

In [30]:
model = Sequential([
          Conv2D(filters = 32, kernel_size=(3,3), activation='relu',input_shape =(224,224,3), padding = 'SAME'),
          MaxPool2D(pool_size=(2,2), strides=2),
          Conv2D(filters = 64, kernel_size=(3,3),activation='relu', padding='SAME'),
          MaxPool2D(pool_size=(2,2), strides=2),
          Flatten(),
          Dense(units = 2, activation='softmax')
])

In [ ]:
model.summary()

In [37]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x = train_batches, epochs=10, validation_data= valid_batches , verbose=2)

Epoch 1/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 2/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 3/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 4/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 5/10
100/100 - 75s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 6/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 7/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 8/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 9/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accuracy: 0.5000
Epoch 10/10
100/100 - 71s - loss: 8.0590 - accuracy: 0.5000 - val_loss: 8.0590 - val_accura

In [ ]:
test_img, test_lables = next(test_batches)
plot_image(test_img)
print(test_lables)

In [ ]:
test_batches.classes

In [42]:
predictions = model.predict(x = test_batches, verbose=0)

In [ ]:
import numpy as np
np.round(predictions)

In [45]:
cm = confusion_matrix(y_true=test_batches.classes ,y_pred=np.argmax(predictions, -1),)

In [46]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
import itertools
cm_plot_lables= ['cats', 'dogs']
plot_confusion_matrix(cm=cm, classes = cm_plot_lables, title = 'Confusion_matrics')